In [1]:
!nvidia-smi

Sun Jan 12 18:56:53 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.39       Driver Version: 418.39       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000035:03:00.0 Off |                    0 |
| N/A   36C    P0    52W / 300W |   3119MiB / 16130MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import torch
import torch.utils.data
import torch.nn as nn
import h5py
import time
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import os

In [3]:
batch_size = 128
Gray = False

In [4]:
if Gray:
    transform_val = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.Grayscale(3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
else:
    transform_val = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

In [5]:
test_dir = './test_images/imagenet_images/'
test_dataset = torchvision.datasets.ImageFolder(test_dir, transform=transform_val)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=8)

In [6]:
#This is the 1000 classes and their labels in the pretrained mobileNet
import json
class_info_json_filename = 'imagenet_class_index.json'
class_info_json_filepath = os.path.join('./', class_info_json_filename)
class_info_dict = dict()

with open(class_info_json_filepath) as class_info_json_f:
    class_info_dict = json.load(class_info_json_f)
for item in class_info_dict:
    class_info_dict[item] = class_info_dict[item][1]
idx2class = {}
for item in class_info_dict:
    idx2class[int(item)] = class_info_dict[item].replace('_',' ')
idx2class

{0: 'tench',
 1: 'goldfish',
 2: 'great white shark',
 3: 'tiger shark',
 4: 'hammerhead',
 5: 'electric ray',
 6: 'stingray',
 7: 'cock',
 8: 'hen',
 9: 'ostrich',
 10: 'brambling',
 11: 'goldfinch',
 12: 'house finch',
 13: 'junco',
 14: 'indigo bunting',
 15: 'robin',
 16: 'bulbul',
 17: 'jay',
 18: 'magpie',
 19: 'chickadee',
 20: 'water ouzel',
 21: 'kite',
 22: 'bald eagle',
 23: 'vulture',
 24: 'great grey owl',
 25: 'European fire salamander',
 26: 'common newt',
 27: 'eft',
 28: 'spotted salamander',
 29: 'axolotl',
 30: 'bullfrog',
 31: 'tree frog',
 32: 'tailed frog',
 33: 'loggerhead',
 34: 'leatherback turtle',
 35: 'mud turtle',
 36: 'terrapin',
 37: 'box turtle',
 38: 'banded gecko',
 39: 'common iguana',
 40: 'American chameleon',
 41: 'whiptail',
 42: 'agama',
 43: 'frilled lizard',
 44: 'alligator lizard',
 45: 'Gila monster',
 46: 'green lizard',
 47: 'African chameleon',
 48: 'Komodo dragon',
 49: 'African crocodile',
 50: 'American alligator',
 51: 'triceratops',
 

In [7]:
#This is the 1000 classes and their labels in the sampled images
class2idx = {} 
for item in test_dataset.class_to_idx:
    #idx2class[item] = class_info_dict[item].replace('_',' ')
    class2idx[test_dataset.class_to_idx[item]] = item


In [8]:
model = torch.hub.load('pytorch/vision:v0.4.2', 'mobilenet_v2', pretrained=True)

model.eval()
model.cuda()

Using cache found in /home/chuqiao8/.cache/torch/hub/pytorch_vision_v0.4.2


MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [9]:
test_accu = []
with torch.no_grad():
    model.eval()
    
    for batch_idx, (X_test_batch, Y_test_batch) in enumerate(test_loader):
        
        correct = 0
        X_test_batch, Y_test_batch= Variable(X_test_batch).cuda(),Variable(Y_test_batch).cuda()

        output = model(X_test_batch)
        
        prediction = output.data.max(1)[1]
        for i,j in zip(prediction,Y_test_batch):
            if int(i.cpu().numpy()) in idx2class and int(j.cpu().numpy()) in class2idx:
                if idx2class[int(i.cpu().numpy())] == class2idx[int(j.cpu().numpy())]:
                    correct += 1/float(batch_size)*100.0
        
        #accuracy = ( float( prediction.eq(Y_test_batch.data).sum() ) /float(batch_size))*100.0
        
        test_accu.append(correct)
        
    test_acc = np.mean(test_accu)

print('len_train_loader', len(test_loader))
print('batch_size:',batch_size)

print('Test Accuraacy = ', test_acc)

len_train_loader 50
batch_size: 128
Test Accuraacy =  77.171875
